In [0]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import WordNetLemmatizer
import sklearn
import codecs
import csv
from keras import optimizers
import logging
import multiprocessing
import gensim
from gensim.models import Word2Vec
from keras.layers import Dense, concatenate, Activation, Dropout
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, HashingVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier, RidgeClassifier, Perceptron
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.svm import LinearSVC
from keras.layers import Input
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
sns.set_style("darkgrid")

from google.colab import drive
drive.mount('/content/drive')


#FUNCTIONS------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------
def list_to_str(arr):
    str_ = ''
    for rec in arr:
        str_+=rec
    return str_

def csv_to_list(arr):
    arr_list = []
    for row in arr:
        arr_list.append(list_to_str(row))
    return arr_list


def value_smiles_in_str(arr,str):
    count = 0
    for smile in arr:
        count+=str.count(smile)
    return count


def mistakes(arr,y,y_pred):
    for i in range(len(arr)):
        if y.iloc[i]!=y_pred[i]:
            print('True: ', y.iloc[i],' Prediction: ', y_pred[i],'\n')
            print(arr.iloc[i])
            
def df_preprocess_str(text):    
    reg = re.compile('[^а-яА-яa-zA-Z0-9 ]') #
    text = text.lower().replace("ё", "е")
    text = text.replace("ъ", "ь")
    text = text.replace("й", "и")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'сайт', text)
    text = re.sub('@[^\s]+', 'пользователь', text)
    text = reg.sub(' ', text)
    #lemmatizer = WordNetLemmatizer()
    text_ = []
    for word in text.split():
        #word = lemmatizer.lemmatize(word)
        text_.append(word)
    return text_


#---------------------------------------------------------------------------------------------------------------

Using TensorFlow backend.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#READ DATA------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------
positive_recalls = csv_to_list(csv.reader(codecs.open('/content/drive/My Drive/Colab Notebooks/positive_recalls.csv', 'rU', 'utf-8', errors='ignore')))
negative_recalls = csv_to_list(csv.reader(codecs.open('/content/drive/My Drive/Colab Notebooks/negative_recalls.csv', 'rU', 'utf-8', errors='ignore')))

columns =['id','tdate','tmane','ttype','trtw','trep','tfav','tstcount','tfol','tfrien','listcount']
positive_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/positive.csv',names=columns, header=None,sep=';')
negative_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/negative.csv',names=columns, header=None,sep=';')

DF_positive = pd.concat((positive_df, pd.DataFrame(positive_recalls,columns= ['text'], index = positive_df.index)), axis = 1)
DF_negative = pd.concat((negative_df, pd.DataFrame(negative_recalls,columns= ['text'], index = negative_df.index)), axis = 1)

DF_positive_ = DF_positive.sample(frac=1.0) # рандомизированный выбор по умолчанию
DF_negative_ = DF_negative.sample(frac=1.0)

df = pd.concat((DF_positive_, DF_negative_),axis = 0).sample(frac = 1) # объединяем и перемешиваем

y = df['ttype'].replace({-1:0, 1:1})
del(df['ttype'])
X_train, X_valid, y_train, y_valid = train_test_split(df, y, test_size=.15, random_state=42, stratify=y)
print(X_train.shape,y_train.shape, X_valid.shape, y_valid.shape)
#---------------------------------------------------------------------------------------------------------------
#DATA PREPROCESSING---------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------

X_train['text'] = X_train['text'].apply(df_preprocess_str)
X_valid['text'] = X_valid['text'].apply(df_preprocess_str)

#---------------------------------------------------------------------------------------------------------------

(192808, 11) (192808,) (34026, 11) (34026,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
SENTENCE_LENGTH = 30

def twitter_share(arr):
  if 2*len(arr) < SENTENCE_LENGTH:
    arr = list(reversed(arr)) + arr
  return arr
X_train['text'] = X_train['text'].apply(twitter_share) 
X_valid['text'] = X_valid['text'].apply(twitter_share) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
#WORD2VEC---------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

text_all = pd.concat([X_train['text'], X_valid['text']], axis = 0)
# Обучаем модель 
model = Word2Vec(text_all, size=200, window=5, min_count=3, workers=multiprocessing.cpu_count())
#model.save("models/w2v/model.w2v")


2019-03-05 07:04:28,394 : INFO : collecting all words and their counts
2019-03-05 07:04:28,406 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-05 07:04:28,466 : INFO : PROGRESS: at sentence #10000, processed 190157 words, keeping 26053 word types
2019-03-05 07:04:28,530 : INFO : PROGRESS: at sentence #20000, processed 380931 words, keeping 42088 word types
2019-03-05 07:04:28,595 : INFO : PROGRESS: at sentence #30000, processed 570889 words, keeping 55125 word types
2019-03-05 07:04:28,659 : INFO : PROGRESS: at sentence #40000, processed 759917 words, keeping 66713 word types
2019-03-05 07:04:28,729 : INFO : PROGRESS: at sentence #50000, processed 949482 words, keeping 77323 word types
2019-03-05 07:04:28,795 : INFO : PROGRESS: at sentence #60000, processed 1139840 words, keeping 87303 word types
2019-03-05 07:04:28,869 : INFO : PROGRESS: at sentence #70000, processed 1330183 words, keeping 96513 word types
2019-03-05 07:04:28,936 : INFO : PROGRESS: a

In [0]:
print(model.wv.most_similar('дима'))
word_vectors = model.wv
words = (word_vectors.vocab.keys())
print(len(words))

2019-03-05 07:05:16,254 : INFO : precomputing L2-norms of word weight vectors


[('леша', 0.7341631054878235), ('дядя', 0.7286885976791382), ('макс', 0.6749944686889648), ('доктор', 0.657037615776062), ('саня', 0.6547166109085083), ('антон', 0.6515439748764038), ('аноним', 0.6511836051940918), ('артур', 0.6509743928909302), ('андреи', 0.6486896276473999), ('билан', 0.6485183835029602)]
71577


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
NUM = len(words)

def get_sequences(tokenizer, x):
    sequences = tokenizer.texts_to_sequences(x)
    return pad_sequences(sequences, maxlen=SENTENCE_LENGTH)

# Cоздаем и обучаем токенизатор
tokenizer = Tokenizer(num_words=NUM)
tokenizer.fit_on_texts(text_all)

# Отображаем каждый текст в массив идентификаторов токенов
x_train_seq = get_sequences(tokenizer, X_train['text'])
x_test_seq = get_sequences(tokenizer, X_valid['text'])

In [0]:
word_vectors = model.wv
words = (word_vectors.vocab.keys())

word_index = tokenizer.word_index

embedding_matrix = np.zeros((len(words) + 1, 200))

#print(model.wv.__getitem__(['олег']).shape)

for word, i in word_index.items():
  if word in words:
    embedding_matrix[i] = model.wv.__getitem__([word])
    #if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
     #   embedding_matrix[i] = embedding_vector

In [0]:
tweet_input = Input(shape=(SENTENCE_LENGTH,), dtype='int32')
tweet_encoder = Embedding(NUM+1, 200, input_length=SENTENCE_LENGTH,
                          weights=[embedding_matrix], trainable=False)(tweet_input)



Instructions for updating:
Colocations handled automatically by placer.


2019-03-05 07:05:40,147 : WARNING : From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# FIRST MODEL--------------------------------------------------------------------
#--------------------------------------------------------------------------------
branches = []
# Добавляем dropout-регуляризацию
x = Dropout(0.25)(tweet_encoder)

for size, filters_count in [(2, 10), (3, 10), (4, 10), (5, 10), (6,10)]: #
    for i in range(filters_count):
        # Добавляем слой свертки
        branch = Conv1D(filters=14, kernel_size=size, padding='same', activation='relu')(x)  #same  valid
        # Добавляем слой субдискретизации
        branch = GlobalMaxPooling1D()(branch)
        branches.append(branch)
# Конкатенируем карты признаков
x = concatenate(branches, axis=1)
# Добавляем dropout-регуляризацию
x = Dropout(0.25)(x)
x = Dense(30, activation='relu')(x)
x = Dense(1)(x)
output = Activation('sigmoid')(x)

model = Model(inputs=[tweet_input], outputs=[output])
#---------------------------------------------------------------------------------

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2019-03-05 07:05:46,829 : WARNING : From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
'''
# SECOND MODEL--------------------------------------------------------------------
#--------------------------------------------------------------------------------
branches = []

sequence_input = Input(shape=(SENTENCE_LENGTH,), dtype='int32')
embedded_sequences = Embedding(NUM+1, 200, input_length=SENTENCE_LENGTH,
                          weights=[embedding_matrix], trainable=False)(tweet_input)

x = Conv1D(filters = 1, kernel_size = (2),padding='valid', activation='relu')(embedded_sequences)
x = GlobalMaxPooling1D()(x)
branches.append(x)
x = Conv1D(filters = 1, kernel_size = (3),padding='valid', activation='relu')(embedded_sequences)
x = GlobalMaxPooling1D()(x)
branches.append(x)
x = Conv1D(filters = 1, kernel_size = (4),padding='valid', activation='relu')(embedded_sequences)
x = GlobalMaxPooling1D()(x)  # global max pooling
branches.append(x)
x = concatenate(branches, axis=1)
#x = Flatten()(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='softmax')(x)

model = Model(inputs=[tweet_input], outputs=[output])


In [0]:
from keras.callbacks import ModelCheckpoint
import os
#checkpoint = ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
#checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)
#checkpoint = ModelCheckpoint(
#    checkpoint_path, verbose=1, save_weights_only=True)
checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/weights-{epoch:02d}.hdf5', monitor='val_acc' , verbose=1, save_best_only=False, mode='max')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 200)      14315600    input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 30, 200)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 30, 14)       5614        dropout_1[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (

In [0]:
history = model.fit(x_train_seq, y_train, batch_size=128, epochs=6, validation_split=0.3, callbacks=[checkpointer])


In [0]:
#model.save("/content/drive/My Drive/Colab Notebooks/CNN_Twitter_2.h5")
# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("/content/drive/My Drive/Colab Notebooks/CNN_Twitter.json", "w")
json_file.write(model_json)
json_file.close()

In [0]:
# ПОСЛЕ ОБУЧЕНИЯ-------------------------------------------------------------------------
#plt.plot(history.history['acc'], label='Доля верных ответов на обучающем наборе')
#plt.plot(history.history['val_acc'], label='Доля верных ответов на проверочном наборе')
#plt.xlabel('Эпоха обучения')
#plt.ylabel('Доля верных ответов')
#plt.legend()
#plt.show()

def y_prep(arr):
    for i in range(len(arr)):
      arr[i] = 0 if arr[i]<0.45 else 1
    return arr

y_p = model.predict(x_test_seq)
y_p = y_prep(y_p)
print("Accuracy on valid data: ", accuracy_score(y_p,y_valid))

Accuracy on valid data:  0.7515429377534826


In [0]:
# ДЛЯ ПРЕОБУЧЕННОЙ МОДЕЛИ--------------------------------------------------
jfile = open("/content/drive/My Drive/Colab Notebooks/CNN_Twitter.json", "r")
loaded_json = jfile.read()
jfile.close()
loaded_model = model_from_json(loaded_json)
loaded_model.load_weights("/content/drive/My Drive/Colab Notebooks/weights-05.hdf5")
loaded_model = model_from_json(loaded_json)
y_p = loaded_model.predict(x_test_seq)
y_p = y_prep(y_p)
print("Accuracy on valid data: ", accuracy_score(y_p,y_valid))

In [0]:
print('Accuracy: ', accuracy_score(y_valid,y_p))
print('Precision: ', precision_score(y_valid,y_p))
print('Recall: ', recall_score(y_valid,y_p))
print('F1: ', f1_score(y_valid,y_p))
print('Confusion matrix:\n', confusion_matrix(y_valid, y_p))

Accuracy:  0.7515429377534826
Precision:  0.7742459251857865
Recall:  0.71926669374021
F1:  0.7457443609022556
Confusion matrix:
 [[13174  3615]
 [ 4839 12398]]


In [0]:
model.save("CNN_Twitter.h5")